## Data Collection for the Recommendation system based on rankings

Author: Karina Condeixa

In [1]:
import sqlite3
import pandas as pd
import urllib.parse
import requests
from urllib.parse import urlparse

import json
import datetime

In [8]:
conn = sqlite3.connect('db.sqlite3')

In [9]:
cursor = conn.cursor()

In [10]:
# Select data from a table
cursor.execute("SELECT * FROM geofree_api_item;")

In [11]:
rows = cursor.fetchall()

In [12]:
df_original = pd.read_sql_query("SELECT * from geofree_api_item", conn)


In [13]:
df = df_original.copy()
df.head()
# df.shape
# df.columns

,id,title,description,available,timedate_creation,latitude,longitude,condition,categories,category_ml,views,likes,point
0,1,test1,wewerwr,1,2023-04-03 17:01:03.428861,52.448102,13.384797,good,"Clothes,Plants,Kitchen",Clothes,0,0,b'\x00\x01\xe6\x10\x00\x00Q4j\x19\x04\xc5*@\x0...
1,2,test1,wewerwr,1,2023-04-03 17:01:21.380176,52.448102,13.384797,good,"Clothes,Kitchen,Plants",Clothes,0,0,b'\x00\x01\xe6\x10\x00\x00Q4j\x19\x04\xc5*@\x0...
2,3,test2,wewerwr,1,2023-04-03 17:01:32.759260,52.448102,13.384797,good,"Kitchen,Plants",Kitchen,0,0,b'\x00\x01\xe6\x10\x00\x00Q4j\x19\x04\xc5*@\x0...
3,4,test4,wewerwr,1,2023-04-03 17:01:41.241430,52.448102,13.384797,good,"Kitchen,Plants,Kids toys",Kitchen,0,0,b'\x00\x01\xe6\x10\x00\x00Q4j\x19\x04\xc5*@\x0...


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 4 non-null      int64  
 1   title              4 non-null      object 
 2   description        4 non-null      object 
 3   available          4 non-null      int64  
 4   timedate_creation  4 non-null      object 
 5   latitude           4 non-null      float64
 6   longitude          4 non-null      float64
 7   condition          4 non-null      object 
 8   categories         4 non-null      object 
 9   category_ml        4 non-null      object 
 10  views              4 non-null      int64  
 11  likes              4 non-null      int64  
 12  point              4 non-null      object 
dtypes: float64(2), int64(4), object(7)
memory usage: 544.0+ bytes


In [15]:
df.tail(20)

,id,title,description,available,timedate_creation,latitude,longitude,condition,categories,category_ml,views,likes,point
0,1,test1,wewerwr,1,2023-04-03 17:01:03.428861,52.448102,13.384797,good,"Clothes,Plants,Kitchen",Clothes,0,0,b'\x00\x01\xe6\x10\x00\x00Q4j\x19\x04\xc5*@\x0...
1,2,test1,wewerwr,1,2023-04-03 17:01:21.380176,52.448102,13.384797,good,"Clothes,Kitchen,Plants",Clothes,0,0,b'\x00\x01\xe6\x10\x00\x00Q4j\x19\x04\xc5*@\x0...
2,3,test2,wewerwr,1,2023-04-03 17:01:32.759260,52.448102,13.384797,good,"Kitchen,Plants",Kitchen,0,0,b'\x00\x01\xe6\x10\x00\x00Q4j\x19\x04\xc5*@\x0...
3,4,test4,wewerwr,1,2023-04-03 17:01:41.241430,52.448102,13.384797,good,"Kitchen,Plants,Kids toys",Kitchen,0,0,b'\x00\x01\xe6\x10\x00\x00Q4j\x19\x04\xc5*@\x0...


In [16]:
df.columns

Index(['id', 'title', 'description', 'available', 'timedate_creation',
       'latitude', 'longitude', 'condition', 'categories', 'category_ml',
       'views', 'likes', 'point'],
      dtype='object')

In [74]:
df.to_csv('data/db_data.csv', index=False)

In [17]:
ranking_df_original = pd.read_sql_query("SELECT * from geofree_api_item", conn)
ranking_df = ranking_df_original.copy()
ranking_df = ranking_df.drop('point', axis=1)


In [18]:
ranking_df['condition'] = ranking_df['condition'].replace('like_new', 1).replace('good', 2).replace('acceptable', 3).replace('poor', 4)

ranking_df['timedate_creation'] = pd.to_datetime(ranking_df['timedate_creation'])
ranking_df['age'] = round((pd.Timestamp.now() - ranking_df['timedate_creation']) / pd.Timedelta(hours=1), 1)


In [19]:
ranking_df

,id,title,description,available,timedate_creation,latitude,longitude,condition,categories,category_ml,views,likes,age
0,1,test1,wewerwr,1,2023-04-03 17:01:03.428861,52.448102,13.384797,2,"Clothes,Plants,Kitchen",Clothes,0,0,116.0
1,2,test1,wewerwr,1,2023-04-03 17:01:21.380176,52.448102,13.384797,2,"Clothes,Kitchen,Plants",Clothes,0,0,116.0
2,3,test2,wewerwr,1,2023-04-03 17:01:32.759260,52.448102,13.384797,2,"Kitchen,Plants",Kitchen,0,0,116.0
3,4,test4,wewerwr,1,2023-04-03 17:01:41.241430,52.448102,13.384797,2,"Kitchen,Plants,Kids toys",Kitchen,0,0,116.0


In [20]:
# ranking_df.head()

In [21]:
url = 'https://geofree.pythonanywhere.com/api/item-user-distance/?user_latitude=52.4480155&user_longitude=13.3832992'


In [22]:
response = requests.get(url)

In [23]:
parsed_url = urlparse(url)
path = parsed_url.path

In [24]:
distance_km = response.json()
distance_km

{'distances': [{'item_id': 1, 'distance_kms': 13.363937847487513},
  {'item_id': 2, 'distance_kms': 0.09577709902078836},
  {'item_id': 3, 'distance_kms': 21.533512674086413},
  {'item_id': 16, 'distance_kms': 4.455127381115968},
  {'item_id': 17, 'distance_kms': 0.10268162264337133}]}

In [25]:
json_str = json.dumps({'distance_km': distance_km})

In [26]:
# Parse the JSON string into a Python object
data = json.loads(json_str)

In [27]:
print(data)

{'distance_km': {'distances': [{'item_id': 1, 'distance_kms': 13.363937847487513}, {'item_id': 2, 'distance_kms': 0.09577709902078836}, {'item_id': 3, 'distance_kms': 21.533512674086413}, {'item_id': 16, 'distance_kms': 4.455127381115968}, {'item_id': 17, 'distance_kms': 0.10268162264337133}]}}


In [28]:
distances = data['distance_km']['distances']

In [29]:
distances_df = pd.DataFrame(distances, columns=['item_id', 'distance_kms'])
distances_df

,item_id,distance_kms
0,1,13.363938
1,2,0.095777
2,3,21.533513
3,16,4.455127
4,17,0.102682


In [30]:
distances_df = distances_df.rename(columns={'item_id': 'id'})

In [31]:
# added to a python file: get-category.py

url_search = 'https://geofree.pythonanywhere.com/api/get-categories/'
response2 = requests.get(url_search)

parsed_url = urlparse(url_search)
path = parsed_url.path

get_categories = response2.json()
get_categories

chosen_category = get_categories[0]['name']
chosen_category

'All'

In [32]:
url_search = 'https://geofree.pythonanywhere.com/api/get-categories/'



In [33]:
response2 = requests.get(url_search)

In [34]:
parsed_url = urlparse(url_search)
path = parsed_url.path

In [62]:
get_categories = response2.json()

get_categories
# get_categories = get_categories.lower()

[{'name': 'All'},
 {'name': 'Furniture'},
 {'name': 'Clothes'},
 {'name': 'Kitchen'},
 {'name': 'Kids toys'},
 {'name': 'Plants'}]

In [64]:
# create a new dictionary to store the modified values
modified_get_categories = []

# loop through the original dictionary
for item in get_categories:
    # create a new dictionary to store the modified value
    modified_item = {}
    # loop through each key-value pair in the original dictionary
    for key, value in item.items():
        # apply the lower method to the value and store it in the modified dictionary
        modified_item[key] = value.lower()
    # add the modified dictionary to the modified dictionary list
    modified_get_categories.append(modified_item)

print(modified_get_categories)

[{'name': 'all'}, {'name': 'furniture'}, {'name': 'clothes'}, {'name': 'kitchen'}, {'name': 'kids toys'}, {'name': 'plants'}]


### Testing

In [67]:
new_data = pd.read_csv('../machine_learning/data/new_data.csv')
new_data

,item_category,item_condition,item_available_timer,distance,n_views,n_likes,item_available
0,kids,poor,7,1222,1813,90,0
1,other,acceptable,50,13107,871,26,1
2,other,acceptable,73,20555,1379,41,1
3,other,like_new,9,5168,825,107,0
4,clothes,like_new,22,15637,1606,96,0
5,other,like_new,62,22332,1267,164,1
6,kids,good,88,11602,871,95,0
7,other,poor,67,6471,938,131,0
8,clothes,acceptable,77,26393,856,51,1
9,other,poor,14,4707,1509,256,1


In [70]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import ndcg_score

In [71]:


# load the digits dataset and split it into train and test sets

X_train, X_test, y_train, y_test = train_test_split(new_data.data, new_data.target, test_size=0.2)

# define the number of folds for cross-validation
k = 5

# initialize the k-fold object
kf = KFold(n_splits=k)

# define the LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)

# define the LightGBM parameters
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'learning_rate': 0.1,
    'max_depth': 3
}

# initialize the ndcg list
ndcg = []

# loop over the folds
for train_idx, test_idx in kf.split(X_train):

    # split the data into train and validation sets
    X_train_fold, X_val_fold = X_train[train_idx], X_train[test_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[test_idx]

    # create the LightGBM dataset for this fold
    train_fold = lgb.Dataset(X_train_fold, label=y_train_fold)

    # train the model on the train set
    model = lgb.train(params, train_fold, num_boost_round=10)

    # evaluate the model on the validation set
    y_pred_val = model.predict(X_val_fold)
    ndcg_fold = ndcg_score(y_val_fold.reshape(1, -1), y_pred_val.reshape(1, -1), k=5)
    ndcg.append(ndcg_fold)

# print the average ndcg score across all folds
print("Average ndcg score:", np.mean(ndcg))

AttributeError: 'DataFrame' object has no attribute 'data'

### Preparing a first new mock dataset for testing [SANDBOX] 

> In order to make it easier of find mock datasets, it was developed in the same jupyter notebook then the one for ML.

In [36]:
# ranking_df_clean = ranking_df.copy()

In [37]:
# ranking_df_clean = ranking_df_clean[ranking_df_clean['categories'] == chosen_category]

# ranking_df_clean['creation_date_rank'] = ranking_df_clean['creation_date'].rank(method='min', ascending=True)
# ranking_df_clean['condition_rank'] = ranking_df_clean['condition'].rank(method='min', ascending=True)
# ranking_df_clean['views_rank'] = ranking_df_clean['views'].rank(method='min', ascending=False)
# ranking_df_clean['likes_rank'] = ranking_df_clean['likes'].rank(method='min', ascending=False)
# ranking_df_clean['age_rank'] = ranking_df_clean['age'].rank(method='min', ascending=True)

# calculate overall rank based on the average of Salary_Rank and Age_Rank
# ranking_df_clean['overall_rank'] = (ranking_df_clean['creation_date_rank'] + ranking_df_clean['condition_rank'] + ranking_df_clean['views_rank'] + ranking_df_clean['likes_rank']) / 5

# ranking_df_clean



In [38]:
 # create a new copy of the df
ranking_df_filtered = ranking_df.copy()

In [39]:
chosen_category = 'Clothes'

In [40]:
ranking_df_filtered


,id,title,description,available,timedate_creation,latitude,longitude,condition,categories,category_ml,views,likes,age
0,1,test1,wewerwr,1,2023-04-03 17:01:03.428861,52.448102,13.384797,2,"Clothes,Plants,Kitchen",Clothes,0,0,116.0
1,2,test1,wewerwr,1,2023-04-03 17:01:21.380176,52.448102,13.384797,2,"Clothes,Kitchen,Plants",Clothes,0,0,116.0
2,3,test2,wewerwr,1,2023-04-03 17:01:32.759260,52.448102,13.384797,2,"Kitchen,Plants",Kitchen,0,0,116.0
3,4,test4,wewerwr,1,2023-04-03 17:01:41.241430,52.448102,13.384797,2,"Kitchen,Plants,Kids toys",Kitchen,0,0,116.0


In [48]:
# filter by categories
ranking_df_filtered = ranking_df_filtered[(ranking_df_filtered['category_ml'] == chosen_category) & (ranking_df_filtered['available'] == 1)]   #|(ranking_df_clean['categories'] in chosen_category)]

# rank these columns 
ranking_df_filtered['timedate_creation_rank'] = ranking_df_filtered['timedate_creation'].rank(method='min', ascending=True)
ranking_df_filtered['condition_rank'] = ranking_df_filtered['condition'].rank(method='min', ascending=True)
ranking_df_filtered['views_rank'] = ranking_df_filtered['views'].rank(method='min', ascending=False)
ranking_df_filtered['likes_rank'] = ranking_df_filtered['likes'].rank(method='min', ascending=False)
# ranking_df_filtered['age_rank'] = ranking_df_filtered['age'].rank(method='min', ascending=True)
ranking_df_filtered['overall_rank'] = (ranking_df_filtered['timedate_creation_rank'] + ranking_df_filtered['condition_rank'] + ranking_df_filtered['views_rank'] + ranking_df_filtered['likes_rank']) / 4
ranking_sorted = ranking_df_filtered.sort_values(by='overall_rank', ascending = False)


In [49]:
ranking_df_filtered

,id,title,description,available,timedate_creation,latitude,longitude,condition,categories,category_ml,views,likes,age,timedate_creation_rank,condition_rank,views_rank,likes_rank,overall_rank
0,1,test1,wewerwr,1,2023-04-03 17:01:03.428861,52.448102,13.384797,2,"Clothes,Plants,Kitchen",Clothes,0,0,116.0,1.0,1.0,1.0,1.0,1.00
1,2,test1,wewerwr,1,2023-04-03 17:01:21.380176,52.448102,13.384797,2,"Clothes,Kitchen,Plants",Clothes,0,0,116.0,2.0,1.0,1.0,1.0,1.25


In [50]:
ranking_sorted = ranking_df_filtered.sort_values(by='overall_rank', ascending = False)
ranking_sorted

,id,title,description,available,timedate_creation,latitude,longitude,condition,categories,category_ml,views,likes,age,timedate_creation_rank,condition_rank,views_rank,likes_rank,overall_rank
1,2,test1,wewerwr,1,2023-04-03 17:01:21.380176,52.448102,13.384797,2,"Clothes,Kitchen,Plants",Clothes,0,0,116.0,2.0,1.0,1.0,1.0,1.25
0,1,test1,wewerwr,1,2023-04-03 17:01:03.428861,52.448102,13.384797,2,"Clothes,Plants,Kitchen",Clothes,0,0,116.0,1.0,1.0,1.0,1.0,1.00


In [51]:
ranked_id = ranking_sorted['id']
ranked_id 

1    2
0    1
Name: id, dtype: int64

In [52]:
# # ranked_id = [2,4]

# # ranked_id = [2]

# # ranked_id = [2,7,9]


# # ranked_id = []
# ranked_id = [2,7,9,8,8,7]

In [53]:
len(ranking_df)

4

In [54]:
import random

In [55]:
# AFTER THE COLUMN ranked_id = ranking_sorted['id']

# if the length of the list is 0, return 3 random ids
if len(ranked_id) == 0: 
    id_1 = random.choice(ranking_df['id'])
    id_2 = random.choice(ranking_df['id'])
    id_3 = random.choice(ranking_df['id'])
    
# if the length of the list is 1, return the id in the list as id_1  and random ids for id_2 and id_3
elif len(ranked_id) == 1: 
    id_1 = list[0]
    id_2 = random.choice(ranking_df['id'])
    id_3 = random.choice(ranking_df['id'])

# if the length of the list is 2, return the ids in the list as id_1, id_2  and random id id_3
elif len(ranked_id) == 2:
    id_1 = list[0]
    id_2 = list[1]
    id_3 = random.choice(ranking_df['id'])

# if the length of the list is 3 or more, return the ids in the list as id_1, id_2, and id_3  
else:
    id_1 = list[0]
    id_2 = list[1]
    id_3 = list[3]
    
id_1 = str(id_1)
id_2 = str(id_2)
id_3 = str(id_3)
    
# return id_1, id_2, id_3   

    

In [56]:
type(id_1)

str

In [57]:
print(id_1)

list[0]


In [58]:
# ranked_id = ranked_id.fillna(0)
# print(ranked_id)

In [59]:
df.columns

Index(['id', 'title', 'description', 'available', 'timedate_creation',
       'latitude', 'longitude', 'condition', 'categories', 'category_ml',
       'views', 'likes', 'point'],
      dtype='object')

In [60]:
## save a init.py file

In [61]:
import numpy as np

description = ['anything', 'everything', 'whaterver', 'hellooo', 'testing', 'bug', 'solving the problem', 'what day is today?', 'today is the presentation day', 'we are the second team to present', 'uhuuuu', 'yeyyyy']
category = ['furniture',  'clothes',  'kitchen_utensils', 'kids', 'other'] 
condition = ['poor', 'acceptable', 'good', 'like_new'] 
available = ['1', '0']  



def data_classification_1(num_records): 
  
    # dictionary 
    item ={} 
    for i in range(0, num_records): 
        item[i] = {}
        item[i]['description']= np.random.choice(description) 
        item[i]['available'] = 1
        item[i]['item_category'] = np.random.choice(category)  
        item[i]['item_condition'] = np.random.choice(condition)
        item[i]['timedate_creation'] = fake.date_between_dates(limit,'now')

#         item[i]['item_available_timer'] = np.random.choice(n_hours_avalilable) # in 30 days
        
#         datetime_iteration1 = fake.date_between_dates(limit,'now')
#         datetime_iteration2 = fake.date_between_dates(limit,'now')
#         if datetime_iteration1 <= datetime_iteration2:
#             item[i]['item_timer'] = datetime_iteration1
#             item[i]['item_timer'] = datetime_iteration2
#         else:
#             item[i]['item_timer'] = datetime_iteration2
#             item[i]['item_timer'] = datetime_iteration1 

        item[i]['item_postcode'] = np.random.choice(postcodes_berlin_series)
    
        item[i]['item_available_timer'] = np.random.choice(n_hours_avalilable)
#         item[i]['item_timer'] = (date.today()) - fake.date_between_dates(limit,'now')  # in days, can be improved to hours
        item[i]['distance'] = np.random.choice(distance)
        
 # This date shold be later than the post
        item[i]['n_views'] = np.random.choice(n_views)
        item[i]['n_likes'] = int(item[i]['n_views'] * (np.random.choice(pct_likes)))
        
        item[i]['item_available'] = np.random.choice(available)
             
    return item